In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [79]:
from src.inference import get_feature_store

In [121]:
from datetime import datetime, timedelta
import pandas as pd

# Get the current datetime64[us, Etc/UTC]
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-02-22 16:51:54,515 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 16:51:54,529 INFO: Initializing external client
2025-02-22 16:51:54,529 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 16:51:55,533 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Fetching data from 2025-01-24 21:51:54.515932+00:00 to 2025-02-22 20:51:54.515932+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.86s) 


In [122]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-24 22:00:00+00:00,2,0
1,2025-01-24 23:00:00+00:00,2,0
2,2025-01-25 00:00:00+00:00,2,0
3,2025-01-25 01:00:00+00:00,2,0
4,2025-01-25 02:00:00+00:00,2,0
...,...,...,...
177325,2025-02-22 11:00:00+00:00,263,130
177326,2025-02-22 12:00:00+00:00,263,152
177327,2025-02-22 13:00:00+00:00,263,143
177328,2025-02-22 14:00:00+00:00,263,111


In [123]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177330 entries, 0 to 183497
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         177330 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  177330 non-null  int32                  
 2   rides               177330 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [124]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [125]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-24 22:00:00,2,0
1,2025-01-24 23:00:00,2,0
2,2025-01-25 00:00:00,2,0
3,2025-01-25 01:00:00,2,0
4,2025-01-25 02:00:00,2,0
...,...,...,...
177325,2025-02-22 11:00:00,263,130
177326,2025-02-22 12:00:00,263,152
177327,2025-02-22 13:00:00,263,143
177328,2025-02-22 14:00:00,263,111


In [126]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177330 entries, 0 to 183497
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         177330 non-null  datetime64[us]
 1   pickup_location_id  177330 non-null  int32         
 2   rides               177330 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.1 MB


In [133]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [134]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-02-03 22:00:00,67
1,14,44,1,44,0,100,17,33,1,1,...,55,37,0,1,10,35,24,42,2025-01-30 10:00:00,13
2,3,2,2,1,4,1,2,2,0,4,...,0,5,5,5,3,3,0,0,2025-02-02 22:00:00,65
3,0,0,0,0,0,0,0,0,0,1,...,1,2,0,1,1,0,0,1,2025-02-06 07:00:00,69
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-02-12 04:00:00,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2025-02-04 07:00:00,98
253,77,2,127,85,53,122,84,128,66,125,...,90,654,21,22,546,63,8,76,2025-02-12 07:00:00,79
254,0,0,0,0,2,2,0,0,2,0,...,3,0,0,4,2,6,2,5,2025-01-31 15:00:00,25
255,0,1,0,0,1,0,1,0,0,0,...,3,0,2,0,0,0,1,0,2025-01-27 11:00:00,242


In [135]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Columns: 674 entries, rides_t-672 to location_id
dtypes: datetime64[ns](1), int32(1), int64(672)
memory usage: 1.3 MB


In [156]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-02-22 17:07:55,087 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:07:55,098 INFO: Initializing external client
2025-02-22 17:07:55,099 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:07:55,868 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Fetching data from 2025-01-24 22:07:55.087298+00:00 to 2025-02-22 21:07:55.087298+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.56s) 


In [148]:
current_date

Timestamp('2025-02-22 21:58:33.356572+0000', tz='Etc/UTC')

In [149]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-02-21 22:00:00,2
1,0,0,0,5,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,2025-02-21 22:00:00,3
2,10,19,42,92,90,48,4,0,0,0,...,2,0,2,3,5,3,5,6,2025-02-21 22:00:00,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-02-21 22:00:00,6
4,0,3,1,0,0,1,2,3,1,0,...,8,3,1,4,1,7,2,2,2025-02-21 22:00:00,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2025-02-21 22:00:00,259
253,0,0,0,0,0,0,0,3,2,1,...,1,2,0,0,0,0,0,0,2025-02-21 22:00:00,260
254,12,15,12,9,5,2,3,3,2,4,...,37,49,40,46,37,31,31,15,2025-02-21 22:00:00,261
255,26,18,10,9,1,6,2,11,14,32,...,78,107,72,104,96,87,72,39,2025-02-21 22:00:00,262


In [150]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-22 16:59:02,420 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 16:59:02,432 INFO: Initializing external client
2025-02-22 16:59:02,432 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 16:59:03,330 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [151]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [152]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,11.0
3,6,0.0
4,7,1.0
...,...,...
252,259,0.0
253,260,0.0
254,261,17.0
255,262,25.0


In [153]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([230, 142,  48, 249,  79, 132, 161, 234, 163, 114])